Packages

In [23]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
import functools
import operator
import pandas as pd
import netCDF4 as nc
import pandas as pd
import xarray as xr
import os

Parameters

In [24]:
Horizon = 2040
start_year = 2024
LeapYearsInvestment = 5
total_periods = int((Horizon - start_year)/LeapYearsInvestment)

pypsa_tech = load_workbook('data_pypsa/technologies.xlsx').active
bus_map = load_workbook('data_pypsa/bus_map30.xlsx').active
new_names = [cell.value for cell in bus_map['A'][1:]]
scale_demand = {
    2025: 1.060816836,
    2026: 1.033372562,
    2027: 1.027658017,
    2028: 1.032454682,
    2029: 1.030974469,
    2030: 1.030564438,
    2031: 1.029585799,
    2032: 1.029366414,
    2033: 1.048001634,
    2034: 1.041580000,
    2035: 1.041580000,
    2036: 1.041580000,
    2037: 1.041580000,
    2038: 1.041580000,
    2039: 1.041580000,
    2040: 1.041580000
}

In [25]:
# Helper functions
def get_new_load(dataframe, columns_to_sum):
    for i, col in enumerate(columns_to_sum):
        while col not in dataframe.columns:
            col = str(int(col)-1)
            columns_to_sum[i] = col
    return dataframe[columns_to_sum].sum(axis=1)

def split_old_buses(old_buses):
    if isinstance(old_buses, str):
        return [bus.strip() for bus in old_buses.split(',')]
    else:
        return [str(old_buses).strip()]  # Handle cases where there's only a single bus name

electricload.csv

In [26]:
electricity_load = pd.read_csv('Data handler/bolivia_v1/ScenarioData/electricload.csv')
pypsa_demand_profiles = pd.read_csv('data_pypsa/demand_profiles.csv')
pypsa_bus_map = pd.read_excel('data_pypsa/bus_map30.xlsx')
bus_numbers_dict = pypsa_bus_map.set_index('Buses')['Gamle noder'].to_dict()

last_three_columns = electricity_load.iloc[:,-4:].copy()
electricity_load = electricity_load.iloc[:,0:0]

for new_bus, old_buses in bus_numbers_dict.items():
    old_buses_list = split_old_buses(old_buses)
    new_values = get_new_load(pypsa_demand_profiles, old_buses_list)
    electricity_load[new_bus] = new_values

electricity_load = pd.concat([electricity_load, last_three_columns.reset_index(drop=True)], axis=1)
electricity_load['time'] = pd.to_datetime(pypsa_demand_profiles['time'])
electricity_load['hour'] = electricity_load['time'].dt.hour
electricity_load['dayofweek'] = electricity_load['time'].dt.dayofweek
electricity_load['month'] = electricity_load['time'].dt.month

# Then change its format to string with a different appearance
electricity_load['time'] = electricity_load['time'].dt.strftime('%d/%m/%Y %H:%M')

electricity_load.to_csv('Data handler/bolivia_v1/ScenarioData/electricload.csv', index=False)

C:\Users\marih\AppData\Local\Temp\ipykernel_12468\3022659369.py:1: DtypeWarning: Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.
  electricity_load = pd.read_csv('Data handler/bolivia_v1/ScenarioData/electricload.csv')


solar.csv

In [27]:
solar = pd.read_csv('Data handler/bolivia_v1/ScenarioData/solar.csv')
weather_path = "data_pypsa/weather_data/"

# Load the solar data
solar_data = xr.open_dataset(weather_path + "profile_solar.nc")
profile_data = solar_data['profile']
profile_df = profile_data.to_dataframe().reset_index()
profile_pivot = profile_df.pivot(index='time', columns='bus', values='profile')
profile_pivot.index = pd.to_datetime(profile_pivot.index)
profile_pivot.reset_index(inplace=True)
profile_pivot.to_csv(weather_path + 'solar_data.csv', index=False)

pypsa_solar = pd.read_csv(weather_path + 'solar_data.csv') 
solar = solar[['time']].copy()
solar['time'] = pd.to_datetime(pypsa_solar['time'],)

# Then change its format to string with a different appearance
solar['time'] = solar['time'].dt.strftime('%d/%m/%Y %H:%M')

for new_bus, old_buses in bus_numbers_dict.items():
    old_buses_list = split_old_buses(old_buses)
    new_values = get_new_load(pypsa_solar, old_buses_list)
    solar[new_bus] = new_values

solar.to_csv('Data handler/bolivia_v1/ScenarioData/solar.csv', index=False)

windonshore.csv

In [28]:
wind = pd.read_csv('Data handler/bolivia_v1/ScenarioData/windonshore.csv')

wind_data = xr.open_dataset(weather_path + "profile_onwind.nc")
profile_data = wind_data['profile']
profile_df = profile_data.to_dataframe().reset_index()
profile_pivot = profile_df.pivot(index='time', columns='bus', values='profile')
profile_pivot.index = pd.to_datetime(profile_pivot.index)
profile_pivot.reset_index(inplace=True)

profile_pivot.to_csv(weather_path + 'wind_data.csv', index=False)

pypsa_wind = pd.read_csv(weather_path + 'wind_data.csv')
wind = wind[['time']].copy()
wind['time'] = pd.to_datetime(pypsa_wind['time'])

# Then change its format to string with a different appearance
wind['time'] = wind['time'].dt.strftime('%d/%m/%Y %H:%M')

for new_bus, old_buses in bus_numbers_dict.items():
    old_buses_list = split_old_buses(old_buses)
    new_values = get_new_load(pypsa_wind, old_buses_list)
    wind[new_bus] = new_values

wind.to_csv('Data handler/bolivia_v1/ScenarioData/windonshore.csv', index=False)

C:\Users\marih\AppData\Local\Temp\ipykernel_12468\750268805.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  wind = pd.read_csv('Data handler/bolivia_v1/ScenarioData/windonshore.csv')


In [29]:
windoff = pd.read_csv('Data handler/bolivia_v1/ScenarioData/windoffshore.csv')

windoff = windoff[['time']].copy()
windoff['time'] = pd.to_datetime(pypsa_wind['time'])

# Then change its format to string with a different appearance
windoff['time'] = windoff['time'].dt.strftime('%d/%m/%Y %H:%M')

# for new_bus, old_buses in bus_numbers_dict.items():
#     old_buses_list = split_old_buses(old_buses)
#     windoff[new_bus] = 0

windoff.to_csv('Data handler/bolivia_v1/ScenarioData/windoffshore.csv', index=False)

hydroror.csv & hydroseasonal.csv

In [30]:
ror_data = pd.read_csv("Data handler/europe_v51/ScenarioData/hydroror.csv")
seasonal_data = pd.read_csv("Data handler/europe_v51/ScenarioData/hydroseasonal.csv")
new_nodes = new_names

time_index = pd.date_range(start='01/01/2013', end='31/12/2013 23:00:00', freq='h')
new_ror = pd.DataFrame(index=time_index, columns=new_nodes)

# Populate the new_ror DataFrame
for column in ror_data.columns[:6]:
    for node in new_nodes:
        new_ror[node] = ror_data[column].values[:8760]

# To work with the index for new columns, use the index directly
new_ror['hour'] = new_ror.index.hour
new_ror['dayofweek'] = new_ror.index.dayofweek
new_ror['month'] = new_ror.index.month

# For saving, if you want 'time' as a column in a specific format
new_ror.reset_index(inplace=True)
new_ror.rename(columns={'index': 'time'}, inplace=True)
new_ror['time'] = new_ror['time'].dt.strftime('%d/%m/%Y %H:%M')

new_ror.to_csv(weather_path + 'hydroror_data.csv', index=False)
new_ror.to_csv("Data handler/bolivia_v1/ScenarioData/hydroror.csv", index=False)

# Repeat the process for seasonal_data
new_seasonal = pd.DataFrame(index=time_index, columns=new_nodes)

for column in seasonal_data.columns[:6]:
    for node in new_nodes:
        new_seasonal[node] = seasonal_data[column].values[:8760]

new_seasonal['hour'] = new_seasonal.index.hour
new_seasonal['dayofweek'] = new_seasonal.index.dayofweek
new_seasonal['month'] = new_seasonal.index.month

new_seasonal.reset_index(inplace=True)
new_seasonal.rename(columns={'index': 'time'}, inplace=True)
new_seasonal['time'] = new_seasonal['time'].dt.strftime('%d/%m/%Y %H:%M')

new_seasonal.to_csv(weather_path + 'hydroseasonal_data.csv', index=False)
new_seasonal.to_csv("Data handler/bolivia_v1/ScenarioData/hydroseasonal.csv", index=False)


General.xlsx

In [31]:
workbook = load_workbook('Data handler/bolivia_v1/General.xlsx')

# seasonScale


# CO2Cap
pypsa_emissions = load_workbook('data_pypsa/emissions.xlsx').active
co2_cap_sheet = workbook['CO2Cap']

emission_year = 2024
start_value = pypsa_emissions['A2'].value
end_year = 2050
end_value = 0

years = range(emission_year, end_year + 1)
emissions_trajectory = []

decrease_per_year = (start_value - end_value) / (end_year - emission_year)

for i in years:
    emission = max(start_value - decrease_per_year * (i - emission_year), end_value)
    emissions_trajectory.append(emission)

for i in range(total_periods):
    co2_cap_sheet.cell(row=i+4, column=1, value= i + 1)
    co2_cap_sheet.cell(row=i+4, column=2, value=emissions_trajectory[i * LeapYearsInvestment])

for i in range(total_periods+4, co2_cap_sheet.max_row + 1):
    for col in range(1, 3): 
        if co2_cap_sheet.cell(row=i, column=col).value is not None:
            co2_cap_sheet.cell(row=i, column=col).value = None

# CO2Price
co2_price_sheet = workbook['CO2Price']
for i in range(total_periods):
    co2_price_sheet.cell(row=i+4, column=1, value= i + 1)
    co2_price_sheet.cell(row=i+4, column=2, value=0)

for i in range(total_periods+4, co2_price_sheet.max_row + 1):
    for col in range(1, 3): 
        if co2_price_sheet.cell(row=i, column=col).value is not None:
            co2_price_sheet.cell(row=i, column=col).value = None

workbook.save('Data handler/bolivia_v1/General.xlsx')

Generator.xlsx

In [32]:
file_path = 'Data handler/bolivia_v1/Generator.xlsx'
# Load the generator workbook
workbook = load_workbook(file_path)

technologies_path = 'data_pypsa/technologies.xlsx'
costs_path = 'data_pypsa/costs_EMPIRE.xlsx'
power_plants_path = 'data_pypsa/custom_powerplants_EMPIRE.xlsx'
generators_path = 'data_pypsa/generators30.xlsx'
generators_empire_path = 'Data handler/europe_v51/Generator.xlsx'

# Function to clear all rows below the header in a sheet
def clear_sheet_rows(ws):
    for row in ws.iter_rows(min_row=4):
        for cell in row:
            cell.value = None
            cell.fill = PatternFill(fill_type=None)
            
# Function to clear all columns to the right of the header in a sheet
def clear_sheet_columns(ws):
    for col in ws.iter_cols(min_col=4):
        for cell in col:
            cell.fill = PatternFill(fill_type=None)


# Load new technologies
new_techs_df = pd.read_excel(technologies_path, sheet_name='technologies_pypsa')
new_technologies = set(new_techs_df['New_names'])

# Load cost data from pypsa
cost_data = pd.read_excel(costs_path, sheet_name='Sheet1')
# Load custom power plant data
power_plants_data = pd.read_excel(power_plants_path, sheet_name='Sheet1')
# Load generator data
generators_data = pd.read_excel(generators_path, sheet_name='Sheet1')
# Load generator rampRate data from empire data
generators_empire_data = pd.read_excel(generators_empire_path, sheet_name='RampRate')


# Map new technology names to old names for lookup
tech_name_map = new_techs_df.set_index('New_names')['Old_names'].to_dict()

tech_name_map2 = new_techs_df.set_index('New_names')['Fueltype_Technology'].to_dict()

# Filter the cost data for investment entries only
investment_cost_data = cost_data[cost_data['parameter'].str.contains('investment', case=False, na=False)]
# Filter the cost data for fixed O&M entries only
FOM_cost_data = cost_data[cost_data['parameter'].str.contains('FOM', case=False, na=False)]
# Filter the cost data for variable O&M entries only
VOM_cost_data = cost_data[cost_data['parameter'].str.contains('VOM', case=False, na=False)]
# Filter the cost data for fuel cost entries only
fuel_cost_data = cost_data[cost_data['parameter'].str.contains('fuel', case=False, na=False)]
# Filter the cost data for CO2 intensity entries only
co2_intensity_data = cost_data[cost_data['parameter'].str.contains('CO2 intensity', case=False, na=False)]
# Filter the cost data for lifetime entries only
lifetime_data = cost_data[cost_data['parameter'].str.contains('lifetime', case=False, na=False)]

# Filter the cost data for efficiency entries only
fueltype_technology = power_plants_data.groupby(['Fueltype', 'Technology']).agg(AverageEfficiency=('Efficiency', 'mean')).reset_index()

efficiency_data = {(f"{row['Fueltype']} {row['Technology']}"): row['AverageEfficiency'] for index, row in fueltype_technology.iterrows()}

# Dictionary to hold the investment values for each new technology
investment_costs = {}
FOM = {}
VOM = {}
fuel = {}
efficiencies = {}
co2_intensities = {}
lifetimes = {}



for new_tech, old_tech in tech_name_map.items():
    tech_cost_data = investment_cost_data[investment_cost_data['technology'].str.lower() == old_tech.lower()]
    if not tech_cost_data.empty:
        investment_costs[new_tech] = tech_cost_data.iloc[0]['value']
    else:
        investment_costs[new_tech] = 0
        
    tech_FOM = FOM_cost_data[FOM_cost_data['technology'].str.lower() == old_tech.lower()]
    if not tech_FOM.empty:
        FOM[new_tech] = tech_FOM.iloc[0]['value']/100 * investment_costs[new_tech]
    else:
        FOM[new_tech] = 0

    tech_VOM = VOM_cost_data[cost_data['technology'].str.lower() == old_tech.lower()]
    if not tech_VOM.empty:
        VOM[new_tech] = tech_VOM.iloc[0]['value']
    else:
        VOM[new_tech] = 0
        
    tech_fuel = fuel_cost_data[cost_data['technology'].str.lower() == old_tech.lower()]
    if new_tech == 'Gas OCGT' or new_tech == 'Gas CCGT':
        tech_fuel = fuel_cost_data[cost_data['technology'].str.lower() == 'gas']
    if not tech_fuel.empty:
        # Change the fuel cost from EUR/MWh to EUR/GJ
        fuel[new_tech] = tech_fuel.iloc[0]['value'] * 3.6
    else:
        fuel[new_tech] = 0
        
    tech_co2 = co2_intensity_data[cost_data['technology'].str.lower() == old_tech.lower()]
    if new_tech == 'Gas OCGT' or new_tech == 'Gas CCGT':
        tech_co2 = co2_intensity_data[cost_data['technology'].str.lower() == 'gas']
    if not tech_co2.empty:
        co2_intensities[new_tech] = tech_co2.iloc[0]['value'] * 3.6
    else:
        co2_intensities[new_tech] = 0
        
    tech_lifetime = lifetime_data[cost_data['technology'].str.lower() == old_tech.lower()]
    if not tech_lifetime.empty:
        lifetimes[new_tech] = tech_lifetime.iloc[0]['value']
    else:
        lifetimes[new_tech] = 0
        
for new_tech, old_tech in tech_name_map2.items():
    # Lowercase the old technology names for case-insensitive matching
    old_tech_lower = old_tech.lower()
    
    # Attempt to find a matching key in efficiency_data (case-insensitive)
    original_key = next((k for k in efficiency_data if k.lower() == old_tech_lower), None)
    
    # Check if a matching key was found
    if original_key:
        # Assign the corresponding efficiency, handling special cases
        if new_tech.lower() in ['hydro regulated', 'hydro run-of-the-river']:
            efficiencies[new_tech] = 1  # Set to 1 or some default value for specific hydro types
        else:
            efficiencies[new_tech] = efficiency_data[original_key]  # Assign the found efficiency
    else:
        # Handle cases where no matching key is found
        efficiencies[new_tech] = 1  # Set to a default value, e.g., 1
    
    
        

for sheet in workbook.sheetnames:
    if sheet in ["CapitalCosts"]:
        ws = workbook[sheet]
        clear_sheet_rows(ws)
        for period in range(1,9):
            for i, (tech, cost) in enumerate(investment_costs.items()):
                ws.cell(row=4 + (period - 1) * len(investment_costs) + i, column=1, value=tech)
                ws.cell(row=4 + (period - 1) * len(investment_costs) + i, column=2, value=period)
                ws.cell(row=4 + (period - 1) * len(investment_costs) + i, column=3, value=cost)
             
                
    elif sheet in ["FixedOMCosts"]:
        ws = workbook[sheet]
        clear_sheet_rows(ws)
        for period in range(1,9):
            for i, (tech, cost) in enumerate(FOM.items()):
                ws.cell(row=4+(period-1) * len(FOM) + i, column=1, value=tech)
                ws.cell(row=4+(period-1) * len(FOM) + i, column=2, value=period)
                ws.cell(row=4+(period-1) * len(FOM) + i, column=3, value=round(cost,3))
                
                
    elif sheet in ["VariableOMCosts"]:
        ws = workbook[sheet]
        clear_sheet_rows(ws)
        for i, (tech, cost) in enumerate(VOM.items()):
                ws.cell(row=4+i, column=1, value=tech)
                ws.cell(row=4+i, column=2, value=cost)
    
    elif sheet in ["FuelCosts"]:
        ws = workbook[sheet]
        clear_sheet_rows(ws)
        for period in range(1,9):
            for i, (tech, cost) in enumerate(fuel.items()):
                ws.cell(row=4+(period-1) * len(fuel) +i, column=1, value=tech)
                ws.cell(row=4+(period-1) * len(fuel) +i, column=2, value=period)
                ws.cell(row=4+(period-1) * len(fuel) +i, column=3, value=cost)
                
        
    elif sheet in ["Efficiency"]:
        ws = workbook[sheet]
        clear_sheet_rows(ws)
        row = 4
        for i, (tech, data) in enumerate(efficiencies.items()):
            for period in range(1,9):
                ws.cell(row=row, column=1, value=tech)
                ws.cell(row=row, column=2, value=period)
                ws.cell(row=row, column=3, value=data)
                row += 1
                
    elif sheet in ["RefInitialCap"]:
        ws = workbook[sheet]
        clear_sheet_rows(ws)
        clear_sheet_columns(ws)
        
        for i, bus in generators_data['bus'].items():
            ws.cell(row=4+i, column=1, value=bus)
            ws.cell(row=4+i, column=2, value=generators_data['carrier'][i])
            ws.cell(row=4+i, column=3, value=generators_data['p_nom'][i])
            if i == 81:
                break
            
    
    
    elif sheet in ["ScaleFactorInitialCap"]: ## HAKKE PEIL
        ws = workbook[sheet]
        clear_sheet_rows(ws)
        for period in range(2,9):
            for i, tech in enumerate(new_technologies):
                ws.cell(row=4+(period-2)*8+i, column=1, value=tech)
                ws.cell(row=4+(period-2)*8+i, column=2, value=period)
                ws.cell(row=4+(period-2)*8+i, column=3, value=0)
                
    elif sheet in ["InitialCapacity"]:
        ws = workbook[sheet]
        clear_sheet_rows(ws)
    
        
    elif sheet in ["MaxBuiltCapacity"]:  ## Usikker her på hva som skal inn, i EMPIREs data er det kun 0 som verdier
        ws = workbook[sheet]
        clear_sheet_rows(ws)
        
        for j, tech in enumerate(new_technologies):
            for period in range(1,9):
                for i, bus in generators_data['bus'].items():
                    ws.cell(row=4+(period-1) * 81 +i, column=1, value=bus)
                    ws.cell(row=4+(period-1) * 81 +i, column=2, value=tech)
                    ws.cell(row=4+(period-1) * 81 +i, column=3, value=period)
                    ws.cell(row=4+(period-1) * 81 +i, column=4, value=0)
                    if i == 81:
                        break
               
    elif sheet in ["MaxInstalledCapacity"]:  ## HARD CODED
        ws = workbook[sheet]
        clear_sheet_rows(ws)
        clear_sheet_columns(ws)
        ws.cell(row=4, column=1, value='Bus')
        
        for j, tech in enumerate(new_technologies):
            tech_max_installed_capacity = {"Geo" : 510, "Bio" : 840}
            for i, bus in generators_data['bus'].items():
                ws.cell(row=4 + i + 81 * j, column=1, value=bus)
                ws.cell(row=4 + i + 81 * j, column=2, value=tech)
                if tech in tech_max_installed_capacity:
                    ws.cell(row=4 + i + 81 * j, column=3, value=tech_max_installed_capacity[tech])
                else:
                    ws.cell(row=4 + i + 81 * j, column=3, value=1000000)
                if i == 81:
                    break
                
    
    elif sheet in ["RampRate"]: ## HARD CODED
        ws = workbook[sheet]
        clear_sheet_rows(ws)
        thermal_gens = ['Gas OCGT', 'Bio', 'Gas CCGT', 'Oil existing', 'Geo']
        ramprates = {'Gas OCGT' : 1.00, 'Bio' : 0.70, 'Gas CCGT' : 0.85, 'Oil existing' : 0.85, 'Geo' : 1.00}
        for i, (tech, rate) in enumerate(ramprates.items()): 
            ws.cell(row=4+i, column=1, value=tech)  
            ws.cell(row=4+i, column=2, value=rate)
            
    elif sheet in ["GeneratorTypeAvailability"]: ## HARD CODED
        ws = workbook[sheet]
        clear_sheet_rows(ws)
        availabilities = {'Gas CCGT' : 1, 'Geo' : 1, 'Wind onshore' : 1, 'Bio' : 0.72, 'Solar' : 1, 'Gas OCGT' : 1, 'Oil existing' : 1, 'Hydro run-of-the-river' : 1, 'Hydro regulated' : 1}
        for i, (tech, avail) in enumerate(availabilities.items()): 
            ws.cell(row=4+i, column=1, value=tech)  
            ws.cell(row=4+i, column=2, value=avail)
                
    elif sheet in ["CO2Content"]:
        ws = workbook[sheet]
        clear_sheet_rows(ws)
        for i, (tech, co2) in enumerate(co2_intensities.items()):
            ws.cell(row=4+i, column=1, value=tech)
            ws.cell(row=4+i, column=2, value=co2)    
                
            
    elif sheet in ["Lifetime"]:
        ws = workbook[sheet]
        clear_sheet_rows(ws)
        for i, (tech, lifetime) in enumerate(lifetimes.items()):
            ws.cell(row=4+i, column=1, value=tech)
            ws.cell(row=4+i, column=2, value=lifetime) 
      
workbook.save(file_path)  

C:\Users\marih\AppData\Local\Temp\ipykernel_12468\2193982268.py:86: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tech_VOM = VOM_cost_data[cost_data['technology'].str.lower() == old_tech.lower()]
C:\Users\marih\AppData\Local\Temp\ipykernel_12468\2193982268.py:92: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tech_fuel = fuel_cost_data[cost_data['technology'].str.lower() == old_tech.lower()]
C:\Users\marih\AppData\Local\Temp\ipykernel_12468\2193982268.py:101: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tech_co2 = co2_intensity_data[cost_data['technology'].str.lower() == old_tech.lower()]
C:\Users\marih\AppData\Local\Temp\ipykernel_12468\2193982268.py:109: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tech_lifetime = lifetime_data[cost_data['technology'].str.lower() == old_tech.lower()]
C:\Users\marih\AppData\Local\Temp\ipykernel_12468\2193982268.py:86: UserWa

Node.xlsx

In [33]:
workbook = load_workbook('Data handler/bolivia_v1/Node.xlsx')
pypsa_load = load_workbook('data_pypsa/load_per_gen30.xlsx').active
load_dict = {row[0]: row[1] for row in pypsa_load.iter_rows(min_row=2, values_only=True)}

for sheet in workbook.sheetnames:
    current_sheet = workbook[sheet]

    nodes_positions = {'A3': ('B3', 1), 'E3': ('F3', 5)}
    
    for node_cell_ref, (period_cell_ref, col_index) in nodes_positions.items():
        if current_sheet[node_cell_ref].value == 'Nodes':
            # Check if the cell next to "Nodes" contains "Period"
            if current_sheet[period_cell_ref].value == 'Period':
                # Logic for handling periods
                start_row = current_sheet[node_cell_ref].row + 1
                
                # Repeat names for each period
                for period_index in range(total_periods):
                    period_start_row = start_row + (period_index * len(new_names))
                    
                    # Fill names for the current period
                    for i, name in enumerate(new_names, start=period_start_row):
                        current_sheet.cell(row=i, column=col_index, value=name)
                        current_sheet.cell(row=i, column=col_index + 1, value=period_index + 1)
                        if period_index == 0:
                            current_sheet.cell(row=i, column=col_index + 2, value=load_dict[name])
                        else:
                            cumulative_scaling_factor = functools.reduce(operator.mul, (scale_demand[start_year + year] for year in range(1,(1+period_index)*LeapYearsInvestment)),1)
                            current_sheet.cell(row=i, column=col_index + 2, value=load_dict[name] * cumulative_scaling_factor)

                # Clear cells after the last period's names, if necessary
                last_name_row = start_row + (total_periods * len(new_names))
                for i in range(last_name_row, current_sheet.max_row + 1):
                    for col in range(1, 8): 
                        if current_sheet.cell(row=i, column=col).value is not None:
                            current_sheet.cell(row=i, column=col).value = None

            else:
                max_row = current_sheet[node_cell_ref].row + len(new_names)
                # Fill names directly if no "Period" is next to "Nodes"
                for i, name in enumerate(new_names, start=current_sheet[node_cell_ref].row + 1):
                    current_sheet.cell(row=i, column=col_index, value=name)

                for i in range(max_row + 1, current_sheet.max_row + 1):
                    for col in range(1, 4): 
                        if current_sheet.cell(row=i, column=col).value is not None:
                            current_sheet.cell(row=i, column=col).value = None

workbook.save('Data handler/bolivia_v1/Node.xlsx')

Sets.xlsx

In [34]:
workbook = load_workbook('Data handler/bolivia_v1/Sets.xlsx')

# Nodes
node_sheet = workbook['Nodes']

max_row = node_sheet['A1'].row + len(new_names)
# Fill names directly if no "Period" is next to "Nodes"
for i, name in enumerate(new_names, start=node_sheet['A1'].row + 1):
    node_sheet.cell(row=i, column=1, value=name)

for i in range(max_row + 1, node_sheet.max_row + 1):
    if node_sheet.cell(row=i, column=1).value is not None:
        node_sheet.cell(row=i, column=1).value = None


# OffshoreNodes
offshore_node_sheet = workbook['OffshoreNodes']
max_row = offshore_node_sheet.max_row

for i in range(1, max_row):
    if offshore_node_sheet.cell(row=i + 1, column=1).value is not None:
        offshore_node_sheet.cell(row=i + 1, column=1).value = None


# Horizon


# Storage
storage_sheet = workbook['Storage']
storage_sheet.cell(row=2, column=1, value='Li-Ion_BESS')
#storage_sheet.cell(row=3, column=1, value='Hydro Pump Storage')
storage_sheet.cell(row=3, column=1).value=None

# Technology
techs = ['Existing', 'Gas', 'Oil', 'Bio', 'Geo', 'Hydro_reg', 'Hydro_ror', 'Wind_onshr', 'Solar']
tech_sheet = workbook['Technology']
for i in range(len(techs)):
    tech_sheet.cell(row=i + 2, column=1, value=techs[i])

# Generators
generators_sheet = workbook['Generators']
for index, row in enumerate(pypsa_tech.iter_rows(min_row=2, values_only=True), start=1):
    generators_sheet.cell(row=index + 1, column=1, value=row[2])

max_row = 1 + len(pypsa_tech['C'])
for i in range(max_row, generators_sheet.max_row + 1):
    if generators_sheet.cell(row=i, column=1).value is not None:
        generators_sheet.cell(row=i, column=1).value = None

thermal_gens = ['Gas OCGT', 'Bio', 'Gas CCGT', 'Oil existing', 'Geo']
for i in range(len(thermal_gens)):
    generators_sheet.cell(row=i + 2, column=4, value=thermal_gens[i])

max_row = 1 + len(thermal_gens)
for i in range(max_row + 1, generators_sheet.max_row + 1):
    if generators_sheet.cell(row=i, column=4).value is not None:
        generators_sheet.cell(row=i, column=4).value = None

# LineType
line_sheet = workbook['LineType']
line_sheet.cell(row=2, column=1, value='HVAC_OverheadLine')
line_sheet.cell(row=3, column=1).value=None

# HourOfSeason


# StorageOfNodes
storage_node_sheet = workbook['StorageOfNodes']
max_row = storage_node_sheet['A3'].row + len(new_names)

for i, name in enumerate(new_names, start=storage_node_sheet['A3'].row + 1):
    storage_node_sheet.cell(row=i, column=1, value=name)
    storage_node_sheet.cell(row=i, column=2, value=storage_sheet['A2'].value)

for i in range(max_row + 1, storage_node_sheet.max_row + 1):
    for col in range(1, 3): 
        if storage_node_sheet.cell(row=i, column=col).value is not None:
            storage_node_sheet.cell(row=i, column=col).value = None

# DirectionalLines
line_connections = workbook['DirectionalLines']
pypsa_connections = load_workbook('data_pypsa/lines30.xlsx').active

for index, row in enumerate(pypsa_connections.iter_rows(min_row=2, values_only=True), start=1):
    value_a, value_b = row[1], row[2]

    line_connections.cell(row=index + 3, column=1, value=value_a)
    line_connections.cell(row=index + 3, column=2, value=value_b)

max_row = 2 + len(pypsa_connections['A'])
for i in range(max_row + 1, line_connections.max_row + 1):
    for col in range(1, 4): 
        if line_connections.cell(row=i, column=col).value is not None:
            line_connections.cell(row=i, column=col).value = None

# LineTypeOfDirectionalLines
line_type_connections = workbook['LineTypeOfDirectionalLines']

for index, row in enumerate(pypsa_connections.iter_rows(min_row=2, values_only=True), start=1):
    value_a, value_b = row[1], row[2]

    line_type_connections.cell(row=index + 3, column=1, value=value_a)
    line_type_connections.cell(row=index + 3, column=2, value=value_b)
    line_type_connections.cell(row=index + 3, column=3, value='HVAC_OverheadLine')

for i in range(max_row + 1, line_type_connections.max_row + 1):
    for col in range(1, 4): 
        if line_type_connections.cell(row=i, column=col).value is not None:
            line_type_connections.cell(row=i, column=col).value = None


# GeneratorsOfNode
generators_node_sheet = workbook['GeneratorsOfNode']
pypsa_generators = load_workbook('data_pypsa/generators30.xlsx').active
for g_index, g_row in enumerate(pypsa_generators.iter_rows(min_row=2, max_row=84 ,min_col=9, max_col=10, values_only=True), start=1):
    generators_node_sheet.cell(row=g_index +3, column=1, value=g_row[1])
    for t_index, tech in enumerate(pypsa_tech.iter_rows(min_row=2, min_col=3, max_col=4,values_only=True), start=1):
        if g_row[0] == tech[1]:
            generators_node_sheet.cell(row=g_index +3, column=2).value = tech[0]
            break
            

max_row = 85
for i in range(max_row + 1, generators_node_sheet.max_row + 1):
    for col in range(1, 3): 
        if generators_node_sheet.cell(row=i, column=col).value is not None:
            generators_node_sheet.cell(row=i, column=col).value = None
    
    

# GeneratorsOfTechnology
generator_tech_sheet = workbook['GeneratorsOfTechnology']
generator_tech_dict = {'Gas OCGT': ['Gas'], 'Gas CCGT': ['Gas'], 'Oil existing': ['Existing', 'Oil'], 'Hydro regulated': ['Hydro_reg'], 'Hydro run-of-river': ['Hydro_ror'], 'Bio': ['Bio'], 'Geo': ['Geo'], 'Wind onshore': ['Wind_onshr'], 'Solar': ['Solar']}
row_num = 4
for key, items in generator_tech_dict.items():
    for item in items:
        generator_tech_sheet.cell(row=row_num, column=2, value=key)
        generator_tech_sheet.cell(row=row_num, column=1, value=item)
        row_num += 1

for i in range(len(generator_tech_dict) + 4, generator_tech_sheet.max_row + 1):
    for col in range(1, 3): 
        if generator_tech_sheet.cell(row=i, column=col).value is not None:
            generator_tech_sheet.cell(row=i, column=col).value = None
    


# Coords
coord_sheet = workbook['Coords']
pypsa_coords = load_workbook('data_pypsa/buses_EMPIRE.xlsx').active

for index, row in enumerate(pypsa_coords.iter_rows(min_row=2, values_only=True), start=1):
    node, x, y = row[0], row[3], row[4]
    coord_sheet.cell(row=index + 3, column=1, value=node)
    coord_sheet.cell(row=index + 3, column=2, value=y)
    coord_sheet.cell(row=index + 3, column=3, value=x)

for i in range(4 + len(new_names), coord_sheet.max_row + 1):
    for col in range(1, 4): 
        if coord_sheet.cell(row=i, column=col).value is not None:
            coord_sheet.cell(row=i, column=col).value = None

workbook.save('Data handler/bolivia_v1/Sets.xlsx')

Storage.xlsx

In [35]:
workbook = load_workbook('Data handler/bolivia_v1/Storage.xlsx')
storage_types = [row[0] for row in storage_sheet.iter_rows(min_row=2, max_col=1, values_only=True) if row[0] is not None]
# InitialPowerCapacity, PowerMaxBuiltCapacity, EnergyInitialCapacity, EnergyMaxBuiltCapacity, EnergyMaxInstalledCapacity, PowerMaxInstalledCapacity

similar_sheets = ['InitialPowerCapacity', 'PowerMaxBuiltCapacity', 'EnergyInitialCapacity', 'EnergyMaxBuiltCapacity', 'EnergyMaxInstalledCapacity', 'PowerMaxInstalledCapacity']

for sheet_name in similar_sheets:
    current_sheet = workbook[sheet_name]

    nodes_cell, period_cell = 'A3', 'C3'
    start_row = current_sheet[nodes_cell].row + 1

    if sheet_name == 'InitialPowerCapacity' or sheet_name ==  'EnergyInitialCapacity':        
        for period_index in range(total_periods):
            period_start_row = start_row + (period_index * len(new_names))
            
            # Fill names for the current period
            for i, name in enumerate(new_names, start=period_start_row):
                current_sheet.cell(row=i, column=1, value=name)
                current_sheet.cell(row=i, column=3, value=period_index + 1)
                # TODO: Update value in columns 'StorageTypes' and 'InitialPowerCapacity'

            # Clear cells after the last period's names, if necessary
        last_name_row = start_row + (total_periods * len(new_names))
        for i in range(last_name_row, current_sheet.max_row + 1):
            for col in range(1, 5): 
                if current_sheet.cell(row=i, column=col).value is not None:
                        current_sheet.cell(row=i, column=col).value = None

    else:
        row_index = 4
        for storage in storage_types:
            for name in new_names:
                current_sheet.cell(row=row_index, column=1, value=name)
                current_sheet.cell(row=row_index, column=2, value=storage)
                if sheet_name == 'PowerMaxBuiltCapacity' or sheet_name == 'EnergyMaxBuiltCapacity':
                    current_sheet.cell(row=row_index, column=3, value=1)
                # TODO: Update value in other columns
                row_index += 1
    
        last_name_row = start_row + (len(storage_types) * len(new_names))
        for i in range(last_name_row, current_sheet.max_row + 1):
            for col in range(1, 5): 
                if current_sheet.cell(row=i, column=col).value is not None:
                        current_sheet.cell(row=i, column=col).value = None


   
# PowerCapitalCost, PowerFixedOMCost, EnergyCapitalCost, EnergyFixedOMCost
similar_sheets = ['PowerCapitalCost', 'PowerFixedOMCost', 'EnergyCapitalCost', 'EnergyFixedOMCost']
for sheet_name in similar_sheets:
    current_sheet = workbook[sheet_name]
    index = 4
    for storage in storage_types:
        for period in range(1, total_periods + 1):
            current_sheet.cell(row=index, column=1, value=storage)
            current_sheet.cell(row=index, column=2, value=period)
            # TODO: Update cost
            index += 1

    for i in range(len(storage_types)*total_periods+4, current_sheet.max_row + 1):
        for col in range(1, 4): 
            if current_sheet.cell(row=i, column=col).value is not None:
                    current_sheet.cell(row=i, column=col).value = None
    

# StorageInitialEnergyLevel, StorageChargeEff, StorageDischargeEff, StoragePowToEnergy, StorageBleedEfficiency, Lifetime
similar_sheets = ['StorageInitialEnergyLevel', 'StorageChargeEff', 'StorageDischargeEff', 'StoragePowToEnergy', 'StorageBleedEfficiency', 'Lifetime']
for sheet_name in similar_sheets:
    current_sheet = workbook[sheet_name]
    index = 4
    for storage in storage_types:
        current_sheet.cell(row=index, column=1, value=storage)
        # TODO: Implement values for column 2 in the different sheets

    for i in range(len(storage_types) + 4, current_sheet.max_row + 1):
        for col in range(1, 4): 
            if current_sheet.cell(row=i, column=col).value is not None:
                    current_sheet.cell(row=i, column=col).value = None

workbook.save('Data handler/bolivia_v1/Storage.xlsx')

Transmission.xlsx

In [36]:
workbook = load_workbook('Data handler/bolivia_v1/Transmission.xlsx')
pypsa_cost = load_workbook('data_pypsa/costs_EMPIRE.xlsx').active

# lineEfficiency, MaxBuiltCapacity, Length, Lifetime
similar_sheets = ['lineEfficiency', 'MaxBuiltCapacity', 'Length', 'InitialCapacity', 'Lifetime']
for sheet in similar_sheets:
    current_sheet = workbook[sheet]
    tracking = 0
    for index, row in enumerate(pypsa_connections.iter_rows(min_row=2, values_only=True), start=1):
        node_1, node_2, distance = row[1], row[2], row[3]
        
        if sheet == 'InitialCapacity':
            row_offset = tracking + 3
            for i in range(1, total_periods + 1):
                period_row = row_offset + i
                current_sheet.cell(row=period_row, column=1, value=node_1)
                current_sheet.cell(row=period_row, column=2, value=node_2)
                current_sheet.cell(row=period_row, column=3, value=i)
                #current_sheet.cell(row=period_row, column=4, value=) TODO: Add initial capacity
        else:    
            current_sheet.cell(row=index + 3, column=1, value=node_1)
            current_sheet.cell(row=index + 3, column=2, value=node_2)
            if sheet == 'lineEfficiency':
                current_sheet.cell(row=index + 3, column=3, value=1) # Line efficiency
            elif sheet == 'MaxBuiltCapacity':
                current_sheet.cell(row=index + 3, column=3, value=1) # Period
                current_sheet.cell(row=index + 3, column=4, value=20000) # Max investement in transmission line, defailt 20 000 MW in Empire
            elif sheet == 'Length':
                current_sheet.cell(row=index + 3, column=3, value=distance) # Distance between nodes
            elif sheet == 'Lifetime':
                    current_sheet.cell(row=index + 3, column=3, value=pypsa_cost['E185'].value) # Lifetime of transmission line

    if sheet == 'InitialCapacity':
        
        for i in range((len(pypsa_connections['A']))*total_periods + 1, current_sheet.max_row + 1):
                for col in range(1, 5): 
                    if current_sheet.cell(row=i, column=col).value is not None:
                        current_sheet.cell(row=i, column=col).value = None    
    else:
        for i in range(3 + len(pypsa_connections['A']), 200):
            for col in range(1, 5): 
                if current_sheet.cell(row=i, column=col).value is not None:
                    current_sheet.cell(row=i, column=col).value = None

# TypeCapitalCost
cost_sheets = ['TypeCapitalCost', 'TypeFixedOMCost']
for sheet in cost_sheets:
    current_sheet = workbook[sheet]
    for i in range(1,total_periods+1):
        current_sheet.cell(row=i+3, column=1, value= line_sheet.cell(row=3, column=1).value)
        current_sheet.cell(row=i+3, column=2, value=i)
        if sheet == 'TypeCapitalCost':
            current_sheet.cell(row=i+3, column=3, value= pypsa_cost['E184'].value)
        elif sheet == 'TypeFixedOMCost':
            current_sheet.cell(row=i+3, column=3, value= pypsa_cost['E186'].value)

    for i in range(total_periods + 4, current_sheet.max_row + 1):
        for col in range(1, 4): 
            if current_sheet.cell(row=i, column=col).value is not None:
                current_sheet.cell(row=i, column=col).value = None


# MaxInstallCapacityRaw



workbook.save('Data handler/bolivia_v1/Transmission.xlsx')